In [19]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_ollama import OllamaLLM, OllamaEmbeddings, ChatOllama
from langchain.chains import RetrievalQA
from IPython.display import Markdown
from langchain_core.prompts import PromptTemplate

In [3]:
# PDF Document Path
document_path = r"C:\Users\Rise Networks\Downloads\whitepaper_Foundational Large Language models & text generation.pdf"

# Load the pdf folder
loader = PyPDFLoader(document_path)
file = loader.load()

In [4]:
# Chunking the loaded pdf document
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 120
)
chunked_document = text_splitter.split_documents(file) 

In [5]:
# Instantiating an embedding model
embedding = OllamaEmbeddings(model="all-minilm")

In [6]:
# Creating a vector database using Chroma
vector_db = Chroma.from_documents(
    documents=chunked_document,
    embedding=embedding,
    # persist_directory="./",
    collection_name="PDF_chat"
)

In [7]:
# Creating a retriever
retriever = vector_db.as_retriever()

In [8]:
# Groq_api_key
os.getenv("GROQ_API_KEY")
# os.environ["GROQ_API_KEY"] = "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"

'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'

In [9]:
# Creating LLM using Groq-AI
llm = ChatGroq(
    name="llama3-70b-8192",
    temperature=0,
    max_retries=2,
    max_tokens=1024
)

In [ ]:
template = """
    You are a question-answering assistant. Your job is to answer a question as accurate as possible. 
    Avoid responding with based on the provided context.

    Human: {user_query}
    Chatbot:
    """
prompt = PromptTemplate(
    input_variables= ["user_query"],
    template=template
)
# prompt = """
#     You are a question-answering assistant. Your job is to answer a question as accurate as possible. 
#     Avoid responding with based on the provided context
#     """

In [23]:
# Chaining all our features together with our retriever
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True,
    chain_type_kwargs={
        "prompt": prompt
    }
)

ValidationError: 1 validation error for StuffDocumentsChain
  Value error, document_variable_name context was not found in llm_chain input_variables: [] [type=value_error, input_value={'llm_chain': LLMChain(ve...None, 'callbacks': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [1]:
response = qa.invoke("Why are large language models important?")

NameError: name 'qa' is not defined

In [19]:
Markdown(response["result"])

Large language models (LLMs) are important because they have significantly improved the performance of natural language processing tasks, and have shown emergent behaviors and zero- or few-shot generalizations to new tasks. This is achieved through the scaling of data and parameters over time. However, despite their advancements, LLMs still have limitations such as not being good at human-like conversations, having limited math skills, and potentially not being aligned with human ethics. These issues are being addressed in the next section. The specific examples provided include GLaM, a model with 1.2 trillion parameters that uses less energy and achieves better performance than GPT-3, and the Chinchilla model, which follows the scaling recommendations of the Kaplan et al.24 study.